In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d nikitarom/planets-dataset

 99% 1.48G/1.50G [00:09<00:00, 204MB/s]
100% 1.50G/1.50G [00:09<00:00, 164MB/s]


In [ ]:
!unzip planets-dataset.zip -d dataset

Streaming output truncated to the last 5000 lines.
  inflating: dataset/test-jpg-additional/test-jpg-additional/file_5499.jpg  
  inflating: dataset/test-jpg-additional/test-jpg-additional/file_55.jpg  
  inflating: dataset/test-jpg-additional/test-jpg-additional/file_550.jpg  
  inflating: dataset/test-jpg-additional/test-jpg-additional/file_5500.jpg  
  inflating: dataset/test-jpg-additional/test-jpg-additional/file_5501.jpg  
  inflating: dataset/test-jpg-additional/test-jpg-additional/file_5502.jpg  
  inflating: dataset/test-jpg-additional/test-jpg-additional/file_5503.jpg  
  inflating: dataset/test-jpg-additional/test-jpg-additional/file_5504.jpg  
  inflating: dataset/test-jpg-additional/test-jpg-additional/file_5505.jpg  
  inflating: dataset/test-jpg-additional/test-jpg-additional/file_5506.jpg  
  inflating: dataset/test-jpg-additional/test-jpg-additional/file_5507.jpg  
  inflating: dataset/test-jpg-additional/test-jpg-additional/file_5508.jpg  
  inflating: dataset/test-jp

In [ ]:
!pip install tensorflow

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import fbeta_score
from tqdm import tqdm
from sklearn.utils import shuffle
import cv2
from PIL import Image
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential 
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, History

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
#reading the labels
train_label = pd.read_csv('../content/dataset/planet/planet/train_classes.csv')
sam_sub = pd.read_csv('../content/dataset/planet/planet/sample_submission.csv')

train_label.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [ ]:
#getting encoded labels
from sklearn.preprocessing import LabelEncoder


encoder = LabelEncoder()

#encoder.fit(train_label['tags'])
label_maps = pd.DataFrame()
label_maps['tags'] = ['agriculture', 'artisinal_mine', 'bare_ground','blooming','blow_down','clear','cloudy','conventional_mine','cultivation','habitation','haze', 'partly_cloudy','primary','road','selective_logging','slash_burn','water']
label_maps['map'] = encoder.fit_transform(label_maps['tags'])
label_maps

,tags,map
0,agriculture,0
1,artisinal_mine,1
2,bare_ground,2
3,blooming,3
4,blow_down,4
5,clear,5
6,cloudy,6
7,conventional_mine,7
8,cultivation,8
9,habitation,9


In [ ]:
#defining a dictionary of encoded labels

label_map = {'agriculture': 0,
 'artisinal_mine': 1,
 'bare_ground': 2,
 'blooming': 3,
 'blow_down': 4,
 'clear': 5,
 'cloudy': 6,
 'conventional_mine': 7,
 'cultivation': 8,
 'habitation': 9,
 'haze': 10,
 'partly_cloudy': 11,
 'primary': 12,
 'road': 13,
 'selective_logging': 14,
 'slash_burn': 15,
 'water': 16}

In [ ]:
#loading the traing_images
X = []
Y = []

train_label = shuffle(train_label,random_state=0)
for image_name, tags in tqdm(train_label.values, miniters=400):
    arr = cv2.imread('../content/dataset/planet/planet/train-jpg/{}.jpg'.format(image_name), cv2.IMREAD_UNCHANGED)
    targets = np.zeros(17)
    for t in tags.split(' '):
      targets[label_map[t]] = 1 
    arr = cv2.resize(arr, (64, 64))
    X.append(arr)
    Y.append(targets)
            
X = np.array(X, np.float16)/255.0

100%|██████████| 40479/40479 [01:05<00:00, 616.21it/s]


In [ ]:
#splitting dataset
from sklearn.model_selection import train_test_split

X = np.array(X)
Y = np.array(Y)
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size = 0.2, shuffle = True, random_state = 1)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(32383, 64, 64, 3) (32383, 17) (8096, 64, 64, 3) (8096, 17)


In [ ]:
#deleting uneccessary datas
import gc
del X, Y
gc.collect()

99

In [ ]:
import gc
gc.collect()

88

In [ ]:
import numpy as np
from tensorflow.keras import backend as K
from sklearn.metrics import fbeta_score

def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 2

    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

In [ ]:
# base model.
from tensorflow.keras.layers import Conv2D, Dense, LSTM, Flatten, MaxPooling2D, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop, SGD


model = keras.Sequential()
model.add(Conv2D(64, 5, 2, activation = "relu", input_shape = (64, 64, 3)))
model.add(MaxPooling2D())
model.add(Conv2D(128, 5, 2, activation = "relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation = "relu"))
model.add(Dense(17, activation = "sigmoid"))
model.compile(loss = "binary_crossentropy", optimizer = Adam(), metrics = [fbeta])
model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 40, batch_size = 500)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 32383 samples, validate on 8096 samples
Epoch 1/40
32383/32383 [==============================] - 86s 3ms/sample - loss: 0.2922 - fbeta: 0.5978 - val_loss: 0.2368 - val_fbeta: 0.5788
Epoch 2/40
32383/32383 [==============================] - 86s 3ms/sample - loss: 0.2245 - fbeta: 0.6107 - val_loss: 0.2096 - val_fbeta: 0.6635
Epoch 3/40
32383/32383 [==============================] - 85s 3ms/sample - loss: 0.2084 - fbeta: 0.6534 - val_loss: 0.2051 - val_fbeta: 0.6446
Epoch 4/40
32383/32383 [==============================] - 85s 3ms/sample - loss: 0.2035 - fbeta: 0.6665 - val_loss: 0.1995 - val_fbeta: 0.6745
Epoch 5/40
32383/32383 [==============================] - 85s 3ms/sample - loss: 0.1960 - fbeta: 0.6840 - val_loss: 0.1913 - val_fbeta: 0.6804
Epoch 6/40
32383/32383 [

In [ ]:
import gc
gc.collect()

782

In [ ]:
test_loss, test_accuracy = model.evaluate(x_val, y_val)
print('Test loss: {}'.format(test_loss))
print('Test accuracy: {}'.format(test_accuracy))

8096/8096 [==============================] - 10s 1ms/sample - loss: 0.1385 - fbeta: 0.7979
Test loss: 0.13851634919525607
Test accuracy: 0.7979146242141724


In [ ]:
import gc
gc.collect()

97

In [ ]:
model.evaluate(x_val, y_val)
print("train fscore: ", fbeta_score(y_train, np.round(model.predict(x_train)), beta = 2,average = 'weighted'))
print("val fscore: ", fbeta_score(y_val, np.round(model.predict(x_val)), beta = 2, average = 'weighted'))

8096/8096 [==============================] - 6s 771us/sample - loss: 0.1385 - fbeta: 0.7979
train fscore:  0.7887471707261128
val fscore:  0.783334976478296


In [ ]:
import gc
del x_train, y_train, x_val, y_val
gc.collect()

167

In [ ]:
#dividing my test_labels into two part for test-jpg and test-jpg-additional
test = sam_sub[0 : 40669]
files = sam_sub[40669 : ]

In [ ]:
test_img = []

for image_name, tags in tqdm(test.values, miniters=1000):
    arr = cv2.imread('../content/dataset/planet/planet/test-jpg/{}.jpg'.format(image_name))
    test_img.append(cv2.resize(arr, (64, 64)))

for image_name, tags in tqdm(files.values, miniters=1000):
    arr = cv2.imread('../content/dataset/test-jpg-additional/test-jpg-additional/{}.jpg'.format(image_name))
    test_img.append(cv2.resize(arr, (64, 64)))

test_img = np.array(test_img, np.float16)/255.0

100%|██████████| 20522/20522 [00:27<00:00, 738.69it/s]


In [ ]:
import gc
gc.collect()

189

In [ ]:
yres = []
predictions = model.predict(test_img, batch_size = 64, verbose = 2)
yres.append(predictions)

61191/61191 - 42s


In [ ]:
#converting my encoded labels back to it original form
sub = np.array(yres[0])
for i in range (1, len(yres)):
    sub += np.array(yres[i])
sub = pd.DataFrame(sub, columns = label_map)

In [ ]:
sub

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,0.025516,8.946657e-05,0.002331,1.712233e-02,1.105040e-03,0.993905,0.001713,0.000114,0.016832,0.003424,0.016856,0.001306,0.991597,0.024668,8.361310e-03,1.121044e-03,0.100226
1,0.316829,4.321337e-06,0.002426,1.110420e-02,4.752129e-03,0.989074,0.000169,0.000046,0.239090,0.016892,0.000682,0.008726,0.999696,0.103660,2.244297e-02,5.250335e-03,0.268111
2,0.103624,9.053946e-05,0.009610,1.877546e-06,2.092123e-05,0.002331,0.004523,0.004413,0.034381,0.025806,0.000103,0.991799,0.996885,0.164528,2.484888e-03,7.980615e-03,0.311702
3,0.134767,9.343028e-05,0.004001,2.022609e-02,7.019639e-03,0.947706,0.000301,0.000159,0.100578,0.010989,0.001323,0.042477,0.998760,0.072824,3.173867e-02,4.896313e-03,0.142887
4,0.147577,7.408857e-05,0.004296,2.384186e-07,1.063943e-05,0.001503,0.146184,0.003301,0.017649,0.045777,0.000148,0.882871,0.835519,0.212930,1.806319e-04,5.877316e-04,0.119259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61186,0.039965,2.522171e-04,0.010019,2.980232e-07,2.563000e-06,0.184053,0.662903,0.005323,0.015023,0.461790,0.000179,0.022800,0.235086,0.526538,3.757477e-04,2.186894e-04,0.090046
61187,0.511500,8.099377e-04,0.093338,1.265496e-03,1.323283e-03,0.994435,0.000067,0.011182,0.098090,0.050777,0.003888,0.002398,0.995203,0.173533,8.398890e-03,6.737947e-03,0.665503
61188,0.206215,1.507998e-05,0.002810,1.774397e-02,5.819499e-03,0.978742,0.000238,0.000072,0.172676,0.012448,0.000927,0.018596,0.999445,0.083240,2.780330e-02,4.934639e-03,0.185972
61189,0.007196,1.490116e-07,0.000029,5.960464e-08,1.192093e-07,0.000645,0.904421,0.000034,0.001538,0.001439,0.185088,0.000809,0.112035,0.007324,8.344650e-07,3.874302e-07,0.097055


In [ ]:
#saving my final result into a csv file
preds = []
for i in tqdm(range(sub.shape[0]), miniters=1000):
    a = sub.loc[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a= a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

sam_sub['tags'] = preds
sam_sub.to_csv('subm.csv', index=False)

100%|██████████| 61191/61191 [02:30<00:00, 406.47it/s]


In [ ]:
sam_sub

,image_name,tags
0,test_0,clear primary
1,test_1,agriculture clear cultivation primary water
2,test_2,partly_cloudy primary water
3,test_3,clear primary
4,test_4,partly_cloudy primary road
...,...,...
61186,file_9995,cloudy habitation primary road
61187,file_9996,agriculture clear primary water
61188,file_9997,agriculture clear primary
61189,file_9998,cloudy
